# 🔄 Pipeline

Load file
Summarize exchanges (currently set to 3 exchanges)
multicatory classification
embedding based classification
rule based classification


# 🛠️ Set environment

In [ ]:
from google.colab import userdata
import os
import openai
import pandas as pd
import json
import time
import re
from tqdm.notebook import tqdm
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
!pip install contractions
!pip install rapidfuzz



# Set secrets
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# ✅ No need to call login() — huggingface_hub will automatically use HF_TOKEN
print("✅ Hugging Face token set.")
openai.api_key = os.environ["OPENAI_API_KEY"]
print("✅ OpenAI API key set.")






from google.colab import drive


# Mount Google Drive
drive.mount('/content/drive')


output_file = "/content/drive/MyDrive/Colab Notebooks/datafiles/Synthetic_ratings.xlsx"  # <-- Set your output file path



✅ Hugging Face token set.
✅ OpenAI API key set.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 📂 Load .xlsx File from Google Drive

<p>When loading your Excel file, make sure the following key variables are included and properly labeled:</p>

<ul>
  <li><strong>ID</strong>: Unique identifier for each <em>therapy session</em>.</li>
  <li><strong>Turn</strong>: A single <em>client–therapist interaction</em>.</li>
  <li><strong>Client</strong>: Text spoken or written by the client.</li>
  <li><strong>Therapist</strong>: Text spoken or written by the therapist.</li>
</ul>


In [ ]:
import pandas as pd
import os

# ✅ Target file name
target_filename = "synthetic transcripts_final.xlsx"

# ✅ Function to search for and load file
def load_file_by_name(filename_exact):
    search_dir = "/content/drive/My Drive"

    for root, dirs, files in os.walk(search_dir):
        for file in files:
            if file.lower() == filename_exact.lower():  # ⬅ exact match
                filepath = os.path.join(root, file)
                print(f"✅ File found: {filepath}")

                if file.endswith((".xlsx", ".xls")):
                    return pd.read_excel(filepath)
                elif file.endswith(".csv"):
                    return pd.read_csv(filepath)
                elif file.endswith(".tsv"):
                    return pd.read_csv(filepath, sep="\t")
                else:
                    raise ValueError(f"⚠️ Unsupported file format: {file}")

    raise FileNotFoundError(f"❌ File named '{filename_exact}' not found in Google Drive.")

# ✅ Load the file into Qdata
Qdata = load_file_by_name(target_filename)
print("✅ Qdata loaded with shape:", Qdata.shape)


✅ File found: /content/drive/My Drive/therapy simulation study 2025/synthetic transcripts_final.xlsx
✅ Qdata loaded with shape: (707, 14)


# **👁️‍🗨️** View orignal Data

In [ ]:
# ✅ Preview the first few rows
display(Qdata.head())
print(f"Total rows: {len(Qdata)}")  # ✅ Preview the first few rows




,ID,Turn,Therapy friendly,Dimension1,Dimension2,Dimension 3,Social dimension,Presenting problem,Intervention Strategy,Therapist approach,Therapist example language,Intended effects,Client,Therapist
0,1,1,Positive Psych,Motivation,NaN,NaN,None explictely simulated,I can't even smile or fake one. I can't feel h...,Use the Miracle Question to help the client im...,"Collaborative, curious, gently persistent. The...","“Suppose tonight, while you are asleep, a mira...",Clarify meaningful goals\nElicit concrete beha...,I just feel numb all the time. Like nothing ma...,It sounds like you're carrying a very heavy we...
1,1,2,NaN,Motivation,NaN,NaN,None explictely simulated,NaN,NaN,NaN,NaN,NaN,Maybe... I don't know. I guess I care enough t...,"That's a significant step, acknowledging that ..."
2,1,3,NaN,Motivation,NaN,NaN,None explictely simulated,NaN,NaN,NaN,NaN,NaN,I... I might not push snooze on my alarm for t...,That's a good start. If you didn't push snooze...
3,1,4,NaN,Motivation,NaN,NaN,None explictely simulated,NaN,NaN,NaN,NaN,NaN,Maybe I'd actually make breakfast instead of j...,Making breakfast could be a sign that the day ...
4,1,5,NaN,Motivation,NaN,NaN,None explictely simulated,NaN,NaN,NaN,NaN,NaN,"I might choose to open the curtains, let some ...",Letting light in could be quite symbolic. How ...


Total rows: 707


# 📚 Classification Systems

fed to later models.

updaed. may 19 2025


Psych_categories; Brief_soc_bio_categories; soc_bio_categories

## Eemm Psych and context classification

need to normalize text before processing so no contractions sent to this

In [ ]:
Psych_categories = {
    "Motivation": """
Focuses on what energizes or drives a person’s actions. This includes:

  - Internal desires such as values, goals, dreams, or personal meaning (e.g., “I want to improve”, “I want to feel proud of myself”),
  - Urges or impulses that compel behavior, even if they do not feel chosen (e.g., “I felt an urge to run away”, “I needed to act on it”),
  - Enjoyment, interest, or preference for certain activities (e.g., “I love painting”, “I enjoy being in nature”, “Cooking helps me relax”),
  - Seeking comfort, pleasure, or calm through specific experiences (e.g., “Listening to music soothes me”, “I watch shows to escape”),
  - External pressures like guilt, obligation, or social expectations (e.g., “I have to do well”, “They expect me to show up”),
  - Lack of drive, disengagement, or disconnection from purpose (e.g., “I don’t care anymore”, “Nothing seems worth the effort”),
  - Wishes or longings that reflect meaningful aspirations or desires for change (e.g., “I wish I were stronger”, “I wish I had direction in life”).

Motivation captures the *why* behind behavior—not the behavior itself. It reflects what matters to the person, what they enjoy or seek out, or what compels them to act—even when the drive is automatic, reactive, or conflicted.

Examples:

- "I want to succeed"
- "I don't care about anything anymore"
- "I value tennis"
- "I feel pressure to do something"
- "I am motivated from guilt"
- "I am driven by my passion for art"
- "I work hard because I want to support my family"
- "I am motivated by the desire to help others"
- "I want to be the best at what I do"
- "I feel a need to prove myself"
- "I want to achieve my goals"
- "I am motivated by fear of failure"
- "I want to make a difference in the world"
- "I feel obligated to meet others' expectations"
- "I am driven by my ambition"
- "I want to escape my current situation"
- "I am motivated by a sense of duty"
- "I want to improve myself"
- "I feel a responsibility to succeed"
- "I want to fulfill my potential"
- "I wish I could stop caring"
- "I wish I had more purpose"
- "I had a strong urge to lash out"
- "I needed to act on the impulse"
- "I enjoy gardening—it helps me feel grounded"
- "I like being alone with my thoughts"
- "I feel better when I’m walking by the ocean"
- "I read to feel connected to something bigger"
- "I love animals—they bring me joy"
- "I bake when I need to calm down"
- "Music motivates me to keep going"
- "I want to feel like I belong"
"""

,

   "Self": """
Focuses on how a person defines, evaluates, and relates to themselves. This includes:
- **Beliefs about the self** (e.g., “I’m not good enough”, “I’m a failure”),
- **Self-concept and identity** (e.g., “I am a good person”, “I am the responsible one”),
- **Self-treatment and social roles** (e.g., “I treat myself with compassion”, “I’m the peacekeeper”),
- **Transcendent or observer self** (e.g., “I am not just my thoughts”, “I am more than my pain”).

Focus is on **statements about the self**, regardless of whether they are affirming, critical, habitual, or aspirational.

Examples:

- "I am a good person"
- "I hate myself"
- "If I fail, then I am a failure"
- "I treat myself with kindness"
- "I treat myself with compassion"
- "I have a spiritual sense of self"
- "I am more than my pain"
- "I see myself as more than just my physical body"
- "I never measure up"
- "I feel like I am an imposter"
- "I am the responsible one"
- "I am a burden"
- "I always let people down"
- "I am a survivor"
- "I am not just my emotions"
- "I am introverted"
- "I am agreeable"
- "I am conscientious"
- "I am open to experience"
- "I am neurotic"
""",

    "Overt Behavior": """
Involves actions that can be observed—or whose **absence can be inferred**—by others. This includes:
- **Intentional actions** (e.g., “I started a conversation”),
- **Habits or routines** (e.g., “I journal every morning”),
- **Avoidant or failed actions** (e.g., “I didn’t respond”),
- **Concealment or inhibition of potential actions** (e.g., “I hid my feelings”),
- **Self-regulatory actions** (e.g., “I made myself go even though I was tired”).

This category focuses on what someone *does* (or deliberately does not do), rather than what they think, feel, or want.

Examples:

- "I exercised today"
- "I avoided talking to them"
- "I asserted myself"
- "I cooked dinner"
- "I cleaned the house"
- "I completed my work tasks"
- "I kept my emotions to myself"
- "I skipped my appointment"
- "I went for a walk to calm down"
- "I stayed in bed all day"
- "I deleted their number"
- "I binge-watched TV all night"
- "I didn’t eat anything all day"
- "I practiced deep breathing"
- "I reached out for help"
- "I stayed quiet even though I wanted to speak"
- "I scrolled on my phone for hours"
- "I cried in front of them"
- "I made a to-do list"
- "I drank too much"
- "I took specific actions to cope"
""",

    "Context": """
Focuses on external or environmental circumstances shaping a person’s experience. This includes:
- **Relationships, social support, and social surroundings** (e.g., “My partner is distant”),
- **Living or working conditions** (e.g., “I live in a noisy neighborhood”),
- **Life events and stressors** (e.g., “I lost my job”),
- **Cultural, institutional, or societal influences** (e.g., “There’s pressure in my community to succeed”),
- **Resources or lack thereof** (e.g., “I don’t have access to therapy”),
- **Physical surroundings* (e.g., “I was in traffic for an hour”).

Focus is on **external background factors**, not internal interpretations or emotional responses.

Examples:

- "I am going through a divorce"
- "My job is very stressful"
- "I recently moved to a new city"
- "I lost my job"
- "I have financial problems"
- "I live close to the park"
- "I grew up in a dysfunctional family"
- "I am being treated for something"
- "I am being helped by others"
- "My mom was cold"
- "My partner travels a lot for work"
- "My parents are getting older"
- "I live in a small apartment with no privacy"
- "I work long shifts at the hospital"
- "There’s been a lot of change at my workplace"
- "My house is under renovation"
- "It’s hard to get childcare"
- "My community doesn’t talk about mental health"
- "I’m living with roommates"
- "We just had a baby"
- "I’ve been in and out of court lately"
- "My dad yells a lot when he's stressed"
"""
,

    "Affect": """
Focuses on ** emotional experiences or feeling states**. This includes:
- **Emotion words** (e.g., sadness, anger, joy, fear, guilt),
- **Emotion-linked physical sensations** (e.g., “my chest tightened”, “heart racing”),
- **Intense emotional language** that implies distress or overwhelm (e.g., “It’s unbearable”, “I’m at my limit”),
- **Attempts to suppress, avoid, or regulate emotions** (e.g., “I try not to feel it”, “I shut it down”).

Affect is **what the person feels**, regardless of whether it's helpful, intense, fleeting, or avoided.

Examples:

- "I feel sad"
- "I am overwhelmed"
- "I feel anxious"
- "I am happy"
- "I am angry"
- "I feel scared"
- "I am excited"
- "I feel guilty"
- "I feel frustrated"
- "I feel calm"
- "I feel stressed out"
- "I feel numb"
- "I cried and couldn’t stop"
- "I just froze"
- "I feel a heaviness in my chest"
- "I try to push the feelings away"
- "I feel helpless"
- "My heart was pounding"
- "I felt relief"
- "I don’t feel anything anymore"
- "It was unbearable"
- "I kept tearing up"
- "I felt peaceful for once"
- "I tried to hold back my emotions"
- "I get choked up when I talk about it"
""",

    "Cognition": """
Focuses on internal thought processes that reflect how a person interprets, judges, or makes sense of things — **excluding beliefs about the self**, which belong in the Self category.

This includes:
- **Beliefs and assumptions** about others, relationships, or the world (e.g., “People can’t be trusted”, “The world is unsafe”),
- **Inferences or mental shortcuts** (e.g., “They did that on purpose”, “It must have been my fault”),
- **Predictions, expectations, and imagined outcomes** (e.g., “This won’t work”, “They’re going to leave me”),
- **Rules and should-statements** (e.g., “People should respect boundaries”, “You should always try your best”),
- **Attitudes and opinions** not about the self (e.g., “Therapy is a waste of time”, “Work is meaningless”),
- **Silent self-talk** or reasoning not tied to identity (e.g., “I need to get this done now”, “This has to go perfectly”).

Focus is on **what and how the person thinks**, not who they are.

Examples:

- "I should have known better"
- "I am thinking about my future"
- "I think people don't like me"
- "I am planning my next steps"
- "I am predicting the outcome"
- "The future is bleak"
- "Others are not to be trusted"
- "I must win"
- "People should not act that way"
- "This always happens"
- "They probably think I’m annoying"
- "That was a mistake"
- "They’re just pretending to be nice"
- "That’s not how things should be"
- "I kept telling myself to stay calm"
- "I figured they were lying"
- "I knew how it would end"
- "They only help people they like"
- "No one really listens"
- "They think they’re better than everyone"
- "There’s no point in trying"
- "Everyone is out for themselves"
- "Life is just random"
- "Rules exist for a reason"
- "People never change"
- "If they cared, they would have called"
- "Therapy is useless"
- "Hard work doesn’t pay off anymore"
- "They were probably just being polite"
""",

   "Attention": """
Focuses on **where attention is directed**, how it moves or gets stuck, and the person's relationship to their focus.
This includes:
- **Rumination and mental looping** (e.g., “I kept going over it in my head”),
- **Preoccupation or attentional narrowing** (e.g., “I couldn’t think about anything else”),
- **Present-moment awareness and mindfulness** (e.g., “I stayed with the sensation”, “I noticed my breath”),
- **Hyperfocus or fixation** (e.g., “I was locked in on every detail”),
- **Avoidance or attempts to shift attention** (e.g., “I tried to distract myself”, “I zoned out on purpose”).

This category captures both **inflexible attention** (dwelling, scanning, stuckness) and **deliberate awareness** (mindful noticing, returning to the present).

It focuses on **how attention operates**—not on the content of thoughts (*Cognition*) or emotions (*Affect*).

Examples:

- "I kept replaying the conversation"
- "I am focused on my work"
- "I can't stop thinking about what happened"
- "I am preoccupied with my thoughts"
- "I am paying attention to my breath"
- "I am mindful of my surroundings"
- "I am worrying too much"
- "I am ruminating"
- "I tried to stay in the moment"
- "My mind kept drifting"
- "I zoned out"
- "I couldn't concentrate"
- "I kept watching the door"
- "I tuned everything else out"
- "I was totally in the zone"
- "I kept going over every detail"
- "I was caught up in my head"
- "I stayed present with the feeling"
- "I noticed every sound around me"
- "I returned my focus to the breath"
- "I noticed thoughts coming and going"
- "I tried to distract myself"
- "I couldn't stop checking"
- "I kept scanning for danger"
- "I just sat there and stared"
"""
}




## Brief EEMM social and bio classificaiton

In [ ]:

Brief_soc_bio_categories = {
    "Relationship / Culture": """
    Focuses on explicit references to social relationships, cultural norms, interpersonal roles, or a sense of group belonging.
    This includes:
    - Family and friendship bonds (e.g., “My family always supports me”),
    - Cultural traditions or values (e.g., “In my culture, we value community”),
    - Social roles and expectations (e.g., “I’m the peacekeeper in my family”),
    - Shared social experiences (e.g., “We had dinner together every night”),
    - Conflict or connection within a social system (e.g., “My parents argued a lot when I was young”).

    Focus is on interactions and relationships with **others**, whether supportive or strained, and on identity within a **social or cultural group**.

    Examples:

    - "My family always supports me"
    - "We had dinner together every night"
    - "In my culture, we value community"
    - "My friends always include me"
    - "I feel a sense of belonging in my group"
    - "My coworkers are very supportive"
    - "My parents argued a lot when I was young"
    - "I am the one everyone depends on"
    - "We were raised to respect our elders"
    - "Our family celebrates everything together"
    - "I was excluded from my friend group"
    - "There is a lot of pressure in my culture to succeed"
    - "We were always taught not to express emotions"
    """,

    "Biology / Physiology": """
    Focuses on physical health, bodily conditions, or medical treatments—distinct from emotions, cognition, or behavior.
    This includes:
    - Chronic or acute health issues (e.g., “I have migraines every day”),
    - Bodily sensations or discomfort (e.g., “My back aches constantly”),
    - Physical diagnoses or medical events (e.g., “I had surgery”),
    - Fatigue, illness, and somatic symptoms (e.g., “I feel exhausted all the time”).

    Emphasis is on **physical functioning**, not mental or emotional states, even when distress is linked to the body.

    Examples:

    - "I’ve had migraines for weeks"
    - "My body aches all over"
    - "I am on dialysis"
    - "I had surgery"
    - "I feel pain in my back"
    - "I have chronic fatigue"
    - "I’ve been nauseous for days"
    - "My joints are inflamed"
    - "I get dizzy if I stand up too fast"
    - "My blood pressure is really high"
    - "I’ve lost weight from the illness"
    - "I’ve been in physical therapy"
    - "I have trouble breathing sometimes"
    """
}

## EEMM social dimensionalized/therapy transcript

In [ ]:
Social_categories = {
    "Relational Content Focus": """
Focuses on how the client describes, navigates, or is shaped by social bonds, roles,
group belonging, or interpersonal meanings. This stance refers to the content of what
is being discussed (e.g., relationships, culture, social support, social identity),
not how the therapist behaves.

Examples:
- "My family expects me to succeed"
- "In my culture, we value elders deeply"
- "I feel like an outsider in most groups"
- "We’ve always done it that way in my community"
- "My friends support me"
- "People see me as the responsible one"
- "Growing up, we weren’t allowed to talk back to adults"
- "I don’t want to disappoint the people who believe in me"
- "I’ve never really felt like I belonged anywhere"
- "They say I’m different because I don’t act like others in my group"
""",

    "Understanding": """
The therapist demonstrates that they understand the client’s internal reality. This
includes active listening, accurate reflections, and a clear grasp of what the client
means both explicitly and implicitly. Understanding does not require agreement or
affirmation—only that the therapist tracks and conveys what the client is experiencing.

Examples:
- "So you're saying it felt like nobody was really there for you."
- "I hear that underneath the anger, there’s hurt."
- "You’re describing a feeling of emptiness even when surrounded by people."
- "It sounds like you were trying hard to keep it together, even when it felt
   impossible."
- "You're saying it wasn’t about what they said—it was how they said it."
- "You felt invisible even when others were physically there."
- "That moment made it feel like everything was spiraling out of control."
- "You were trying to stay strong, but inside it felt like you were falling apart."
- "It felt like nobody really understood what you needed in that moment."
- "Even when you asked for help, it didn’t land the way you hoped."
""",

    "Interpersonal effectiveness": """
The therapist displays warmth, concern, confidence, and genuineness while maintaining
appropriate professionalism. This stance also includes validating the client’s internal
experience and offering realistic hope or encouragement when appropriate. The therapist
remains emotionally available without being overfamiliar, aloof, or detached.

Examples:
- "That makes total sense."
- "Anyone would feel overwhelmed in your shoes."
- "Of course it was hard to speak up after what happened."
- "It’s understandable that you shut down in that moment."
- "That kind of stress would push anyone past their limit."
- "You’ve gotten through things like this before—you have that strength."
- "It’s okay to feel like this, and you’re not alone."
- "I really admire how open you're being right now."
- "It makes sense you'd feel torn—it’s a tough situation."
- "You’ve held a lot together—more than most people might realize."
- "I believe in your ability to move through this."
- "You’ve shown so much resilience in how you’ve handled all of this."
- "You matter, even when it doesn’t feel like it."
- "It’s valid to feel both strong and overwhelmed at the same time."
""",

    "Collaboration": """
The therapist invites the client to participate actively in planning, reflection, or
goal setting. This stance emphasizes mutual decision-making and shared agency. It
includes seeking permission before introducing challenging interventions, offering full
disclosure about what those steps involve, and checking the client's readiness to
proceed.

Examples:
- "What feels most important to explore today?"
- "How would you like to approach this?"
- "Let’s figure this out together."
- "We can come up with a plan as a team."
- "Do you feel ready to try this together?"
- "What would make this feel more useful for you?"
- "What’s the next step that feels doable for us both?"
- "Would it be okay if we revisited this together later?"
- "Are there ways you’d prefer we handle moments like that in session?"
- "Tell me what works best for you when we’re problem-solving."
- "I have a suggestion, but I want to check if you’re open to trying something a bit
   uncomfortable."
- "This might be a tough conversation—are you okay if we go there together?"
- "Before we start, I want to be clear about what this process might bring up for you."
- "We can pause or stop at any point—this is entirely up to you."
""",

    "Ineffective": """
The therapist uses strategies that are poorly timed, overly directive, or emotionally
misattuned. They may miss emotional cues, shift focus too quickly, or fail to validate
the client’s experience.

Examples:
- Therapist interrupts a moment of emotion to ask about logistics
- Therapist insists on a technique the client resists
- Therapist fails to reflect or acknowledge the client’s distress
- Therapist appears disengaged or uncertain
- Therapist gives advice prematurely or relies on generic phrasing
- Therapist responds to emotional disclosure with a checklist
- Therapist rushes into interpretation without hearing the full story
- Therapist overexplains instead of listening
- Therapist misses a key emotional cue and redirects the topic
- Therapist says, "Let's not dwell on that" while the client is still upset
"""
}


## social simplified

In [ ]:
Brief_Social_categories = {
    "Relational Content Focus": """
Focuses on the content of what the client talks about—specifically, how their identity,
experience, and meaning-making are shaped by social relationships, cultural norms, and
group belonging. This category is about **what is discussed**, not how the therapist
responds.
""",

    "Understanding": """
Refers to the therapist’s ability to track, reflect, and grasp the client’s internal
reality—both emotional and cognitive—without needing to agree or affirm. This is about
**what the therapist understands and conveys**.
""",

    "Interpersonal effectiveness": """
Describes the therapist’s ability to express warmth, professionalism, and grounded
emotional presence. Includes validation, encouragement, and the ability to stay engaged
without becoming distant or overfamiliar. This is about **how the therapist shows up**.
""",

    "Collaboration": """
Captures the therapist’s efforts to engage the client as an active partner. Includes
shared planning, seeking permission for difficult work, and ensuring transparency.
This is about **how the therapist involves the client**.
""",

    "Ineffective": """
Refers to therapist behaviors that are poorly timed, misattuned, overly directive,
or emotionally disengaged. Often involves missed cues, dismissiveness, or impersonal
responses. This is about **therapist actions that disrupt therapeutic connection**.
"""
}


## EEMM bio dimensionalized

In [ ]:
bio_categories = {
    "Somatic / Bodily Functioning": """
Refers to physical health, pain, medical issues, or bodily sensations and processes.
Focus is on physical functioning, not emotional or mental states.

Examples:
- "I’ve had migraines for weeks"
- "My body aches all over"
- "I am on dialysis"
- "I had surgery"
- "I feel pain in my back"
- "I have chronic fatigue"
- "My joints feel stiff every morning"
- "I get dizzy when I stand up"
- "My stomach has been upset all day"
- "I’m dealing with a lot of physical symptoms lately"
""",

    "Exercise": """
Describes intentional physical activity involving movement or exertion.
Must involve actual physical action, not just intent or planning.

Examples:
- "I exercised today"
- "I went for a walk"
- "I did yoga this morning"
- "I danced for an hour"
- "I followed my physical therapy routine"
- "I ran when I was stressed"
- "I lifted weights at the gym"
- "I swam laps at the pool"
- "I biked around my neighborhood"
- "I stretched after sitting too long"
""",

    "Diet": """
Refers to eating habits, nutritional choices, or food-related behaviors.
Focuses on what and how someone eats.

Examples:
- "I stopped eating sugar"
- "I eat when I’m upset"
- "I follow a balanced diet"
- "I binge eat sometimes"
- "I restrict my food intake"
- "I am conscious of my nutrition"
- "I skip meals often"
- "I started eating more vegetables"
- "I try to avoid processed food"
- "I track my calorie intake"
""",

    "Sleep": """
Describes sleep patterns, sleep quality, or sleep-related issues.
Must involve actual sleep experiences.

Examples:
- "I barely sleep at night"
- "I wake up every hour"
- "I can’t fall asleep"
- "I slept better this week"
- "I have insomnia"
- "I have a regular sleep schedule"
- "I take naps during the day"
- "I wake up feeling exhausted"
- "I get anxious before bed"
- "I dream a lot when I sleep"
""",

    "Ingestion of Substances": """
Refers to consumption of non-food substances like alcohol, drugs, medications, caffeine, or supplements.
Focuses on how substances affect the body or mind.

Examples:
- "I drank to calm down"
- "My meds make me tired"
- "I take supplements for my health"
- "I avoid caffeine"
- "I use drugs recreationally"
- "I consume a lot of sugar"
- "I smoked weed last night"
- "I need coffee to get through the day"
- "I took something to help me sleep"
- "I use nicotine when I’m stressed"
"""
}


## Simplified process-focused categories: Psychological and context

In [ ]:
ProcessFocusCategories = {
      "Motivation": (
        "The therapist focuses on what energizes or drives the client’s actions. This includes internal desires such as values, "
        "goals, dreams, or personal meaning (e.g., 'I want to improve'), as well as urges or impulses that compel behavior, even "
        "if they don’t feel chosen (e.g., 'I needed to act on it'). The therapist may help the client notice, understand, and manage "
        "these urges without judgment. Motivation can also involve external pressures like guilt, obligation, or social expectations, "
        "a lack of drive or disengagement, and wishes or longings that reflect meaningful aspirations or a desire for change."
    ),
    "Self": (
        "The therapist helps the client examine or change how they relate to themselves — including how they see, "
        "evaluate, and treat themselves. This includes identity, self-worth, internal roles, and patterns of "
        "self-treatment such as self-compassion, self-criticism, neglect, or care. The focus may involve exploring shame, "
        "internalized labels, or how the client responds to their own suffering."
    ),
    "Cognition": (
        "The therapist helps the client explore and work with their thoughts — such as beliefs about others, expectations, "
        "assumptions, or internal dialogue. These thoughts are not about the client's identity or self-worth, but about how "
        "they interpret the world, situations, or future events. Strategies may include reappraisal, defusion, problem-solving, "
        "or shifting perspective."
    ),
    "Affect": (
        "The therapist is helping the client feel, express, or work with their emotions. This may involve identifying emotions, "
        "exploring emotional experiences, processing distress, or regulating strong feelings through grounding or acceptance. "
        "The focus is on emotional experience — not the thoughts about it or the actions it causes."
    ),
    "Attention": (
        "The therapist is helping the client notice, redirect, or relate differently to where their attention goes. This includes "
        "increasing present-moment awareness, addressing rumination or worry, or practicing mindful shifts of focus."
    ),
    "Overt Behavior": (
        "The therapist works on behavior change — such as action planning, building habits, reducing avoidance, activating behavior, "
        "or reinforcing new actions."
    ),
    "Context": (
        "The therapist focuses on external influences or circumstances — including relationships, social support, work, culture, "
        "stressors, or access to resources."
    )
}


## keyword sets for rule-based psych classification

In [1]:
Psych_keywords = {
   "Motivation": [

    # Value-based or purposeful motivation
    "value", "values", "purpose", "goal", "goals", "dream", "dreams", "aspiration", "ambition",
    "meaningful", "meaning", "I care about", "it matters to me", "care deeply",
    "I want to make a difference", "I want to fulfill my potential", "I want to improve myself",
    "I wish I had more purpose", "I wish I could find purpose", "I want to be the best",

    # Commitment and dedication
    "committed to", "dedicated to", "passionate about", "I work hard because",
    "I want to support my family", "I am motivated", "I am driven", "I am motivated by duty",

    # Desire, drive, or longing
    "want to", "wish", "hope", "desire", "need to", "long to", "I want to succeed",
    "I want to achieve", "I need to prove myself", "I want to escape", "I wish I were stronger",
    "I do not care anymore", "I wish I could stop caring",

    # Felt pressure and obligation
    "have to", "must", "ought to", "should", "pressure to", "obligation", "I feel obligated",
    "I feel a responsibility",

    # Change-oriented motivation
    "feel like changing", "change my life", "I want to change", "I want to grow",
    "I want to do better", "I want to improve",

    # Motivational emotions
    "motivated", "determined", "driven", "inspired",

    # Urges and impulses (motivational drives)
    "urge", "urges", "I felt an urge", "I had the urge to", "craving", "I craved",
    "I wanted to lash out", "I wanted to scream", "I had an impulse", "compelled to",
    "urge to act", "urge to avoid"
],

    "Self": [
        # Identity and self-concept
        "identity", "who I am", "how I view myself", "self-image", "self-worth", "self-esteem",
        "I define myself", "I see myself", "I am the kind of person", "I am the type of person",
        "I am not just my emotions", "I have a spiritual sense of self",

        # Self-evaluation (positive/neutral)
        "I love myself", "self-compassion", "I am a survivor", "I am more than my pain",

        # Self-criticism and low self-concept
        "self-criticism", "I feel like a failure", "I am worthless", "I am not good enough",
        "I hate myself", "I do not like myself", "I always let people down", "I am a burden",
        "I feel like I am an imposter",

        # Self-treatment and self-reflection
        "I treat myself", "self-image", "self-esteem", "how I view myself"
    ],

    "Overt Behavior": [
        # Positive Coping
        "I exercised", "I cleaned the house", "I cooked dinner", "I completed my work tasks",
        "I practiced deep breathing", "I made a to-do list", "I journaled", "I went to bed early",
        "I got out of bed", "I took deep breaths", "I made myself go", "I kept busy",

        # Maladaptive Coping
        "I binge-watched TV", "I drank too much", "I didn’t eat", "I binged", "I shut down",
        "I stayed in bed", "I scrolled", "I cried in front of them", "I cleaned obsessively",
        "I canceled my plans", "I isolated", "I froze", "I lay down", "I held in my tears",

        # Avoidance
        "I avoided", "I avoided the conversation", "I avoided eye contact", "I withdrew",
        "I ignored", "I canceled", "I bit my tongue", "I stayed quiet", "I shut the door",
        "I turned off my phone", "I stayed out of the way", "I held back", "I packed my things",
        "I kept quiet", "I suppressed my reaction", "I kept my emotions to myself",

        # General Action
        "I reached out", "I called", "I responded", "I texted", "I left", "I went", "I stayed",
        "I yelled", "I ran", "I walked", "I confronted", "I followed through", "I skipped",
        "I started", "I acted on", "I made a decision", "I took action", "I coped by"
    ],

    "Affect": [
    # Direct self-statements of emotional state
    "I cried", "I feel grief", "I feel guilty", "I feel joy", "I feel peaceful",
    "I feel relieved", "I feel shame", "I felt", "I felt hopeless", "I was anxious",
    "I was elated", "I was happy", "I was heartbroken", "I was numb", "I was scared",
    "I am afraid", "I am angry", "I am content", "I am devastated", "I am discouraged",
    "I am excited", "I am feeling", "I am frustrated", "I am irritable", "I am panicked",
    "I am relaxed", "I am sad", "I am tense", "I am uneasy",

    # Emotion-related nouns and phrases
    "anger", "anxiety", "ashamed", "blue", "butterflies", "calm", "cheerful", "choking up",
    "contentment", "crying", "distress", "downhearted", "drowsy", "edgy", "emotional",
    "emotions", "enthusiastic", "euphoric", "excited", "fear", "felt emotional", "felt empty",
    "felt intense", "felt numb", "flat", "frustration", "grief", "guilt", "happiness",
    "heart racing", "heaviness", "hostile", "irritability", "jittery", "joy", "lively",
    "loneliness", "melancholy", "moody", "numbness", "on edge", "peacefulness", "proud",
    "relief", "restless", "sadness", "scared", "shaky", "shame", "sheepish", "tears",
    "tension", "tight chest", "timid", "tired", "touchy", "trembled", "unsettled", "upset",
    "weepy",

    # Combined physical-emotional cues
    "angry at self", "alone", "lump in throat", "could not stop crying"
],

    "Context": [
        # Relationships and social surroundings
        "my partner", "my spouse", "my friend", "my parents", "my family", "my sibling", "my child",
        "my kids", "my roommate", "my colleagues", "my coworkers", "people around me",

        # Life events and stressors
        "divorce", "breakup", "lost my job", "got fired", "laid off", "financial issues",
        "financial stress", "debt", "foreclosure", "court case", "custody battle",
        "recently moved", "new job", "new school", "death in the family", "bereavement",
        "hospitalized", "hospital visit", "rehab", "accident", "disaster", "natural disaster",
        "out of work", "just had a baby", "caregiving responsibilities",

        # Living and working conditions
        "noisy neighborhood", "unsafe neighborhood", "small apartment", "shared housing",
        "poor living conditions", "long shifts", "workplace stress", "bad work environment",
        "under renovation", "crowded home", "limited space", "long commute", "traffic",
        "unpredictable schedule", "lack of privacy",

        # Cultural, institutional, societal
        "social pressure", "cultural pressure", "community expectations", "discrimination",
        "racism", "sexism", "institutional bias", "school rules", "government policy",
        "legal system", "immigration system", "visa issues", "cultural norm", "religious rule",
        "healthcare system", "educational system", "university demands", "academic pressure",
        "workplace politics", "military service", "legal issue", "court system",

        # Resources and access
        "lack of support", "can’t afford", "low income", "poverty", "unemployment",
        "underemployment", "can’t get childcare", "can’t access therapy", "limited resources",
        "transportation issues", "no health insurance", "waiting list", "unavailable services",

        # Environmental/temporal
        "weather", "heatwave", "cold snap", "holiday season", "it's the holidays",
        "school holidays", "exam period", "moving day", "renovation", "rush hour",
        "bedtime", "end of the month", "seasonal", "shift work", "late at night",
        "weekends", "weekday", "morning", "evening"
    ],

    "Cognition": [
        # Beliefs, assumptions, and judgments
        "I think", "I believe", "I assumed", "I figured", "I concluded", "I reasoned",
        "I interpreted", "I expected", "I judged", "I imagined", "I guessed", "I predicted",
        "I made sense of", "I thought it meant", "it means that", "this must mean",
        "I told myself", "my assumption", "my perspective", "my thoughts",

        # Inferences and attributions
        "they probably", "they must be", "they did that on purpose", "they’re just pretending",
        "they only help people they like", "they think they’re better than everyone",
        "others are not to be trusted", "everyone is out for themselves",

        # Rules, attitudes, and opinions
        "should", "should have", "must be", "have to be", "rules about", "that’s not how things should be",
        "people should", "rules exist for a reason", "therapy is useless", "work is meaningless",

        # Predictions and imagined outcomes
        "I am planning", "I am predicting", "I am imagining", "I knew how it would end",
        "this won’t work", "they’re going to leave me", "the future is bleak",
        "life is just random", "hard work doesn’t pay off anymore",

        # Silent self-talk and reasoning
        "I kept thinking", "kept telling myself", "I need to", "I have to", "I must",
        "I should have known better", "I need to get this done now", "this has to go perfectly"
    ],

    "Attention": [
        # Attentional focus and control
        "attention", "focused", "focus", "concentrate", "concentration", "staying focused",
        "struggling to focus", "hard to concentrate", "easily distracted", "attention wandered",

        # Internal distraction or fusion
        "ruminate", "rumination", "worry", "worried", "obsess", "obsessed", "dwelling",
        "looping", "overthinking", "in my head", "thinking about", "kept thinking",
        "kept going over", "could not stop thinking", "mentally stuck", "stuck in my head",
        "replaying what happened", "going over it again and again",

        # External distraction or inattention
        "distracted", "tuned out", "zoned out", "mind wandered", "drifted", "spacing out",
        "lost track of what was happening", "not paying attention",

        # Mindfulness and noticing
        "mindful", "mindfulness", "present", "present-moment", "noticing", "noticed",
        "observed", "observe", "aware", "awareness", "noticed my breath",
        "noticed sensations", "noticed thoughts", "grounded in the moment",
        "brought myself back to the moment",

        # Hypervigilance and scanning
        "scanning", "hypervigilant", "on high alert", "mentally scanning",
        "looking out for signs", "watching for reactions", "constantly checking"
    ]
}


## keywords for social catagories

In [ ]:


Social_keywords = {
    "Relational Content Focus": [
        "my family", "my parents", "my siblings", "my children", "my kid", "my partner", "my spouse",
        "my husband", "my wife", "my friends", "my coworkers", "my colleagues", "people I know", "my community",
        "people close to me", "people around me", "in my culture", "in our culture", "cultural norms",
        "our traditions", "our customs", "how I was raised", "upbringing", "family expectations",
        "community expectations", "social expectations", "social roles", "social pressure", "they expect me to",
        "they want me to", "they believe I should", "group norms", "group belonging", "feeling excluded",
        "I don’t fit in", "I’ve never belonged", "I feel like an outsider", "I’ve always felt different",
        "I feel misunderstood", "people rely on me", "they see me as", "they believe in me",
        "social identity", "social support", "relationships", "connection with others", "belonging to a group"
    ],

    "Understanding": [
        "so you're saying", "it sounds like", "what I'm hearing is", "I hear that", "underneath that",
        "what you’re describing", "you're describing", "you felt like", "you feel like", "felt invisible",
        "felt unseen", "nobody was there for you", "you’re saying it wasn’t about", "it wasn’t about what happened",
        "you were trying to hold it together", "it felt impossible", "it sounds like you were",
        "you needed support", "nobody understood you", "nobody seemed to get it", "it didn’t land",
        "you felt dismissed", "you felt unheard", "you felt overlooked", "felt empty", "everything was spiraling",
        "it felt chaotic", "felt like falling apart", "you tried to stay strong inside",
        "even when you asked for help", "it didn’t feel like anyone cared", "you felt shut down"
    ],

    "Interpersonal Effectiveness": [
        "that makes sense", "of course it was hard", "it’s understandable you felt",
        "anyone would feel that way", "it’s okay to feel this way", "you’re not alone in this",
        "you’ve gotten through hard things", "you have the strength to", "I admire your openness",
        "I believe in your ability", "it’s valid to feel this", "you matter", "your feelings matter",
        "you’ve shown resilience", "I really respect how", "I appreciate your honesty",
        "it's normal to feel like this", "you're doing your best", "you’ve been carrying so much",
        "you’re doing great under the circumstances", "you’ve held it together", "I trust your instincts",
        "it’s natural to feel torn", "you’ve coped with a lot", "it’s human to feel overwhelmed"
    ],

    "Collaboration": [
        "what feels most important", "do i have permission to talk about this", "how would you like to approach", "let’s figure this out together",
        "we can come up with a plan", "we’ll work together on this", "do you feel ready to try this",
        "what would help this feel more useful", "what’s the next step for you", "would it be okay if",
        "are there ways you’d prefer we handle", "tell me what works best for you", "I have a suggestion if you're open",
        "are you open to trying", "this might be challenging, are you okay with",
        "are you comfortable if we go there", "before we start, I want to be clear", "this is entirely your choice",
        "we can pause at any time", "you can stop this whenever you need", "this is collaborative",
        "let’s decide together", "let’s plan this out together", "what would make this easier for you",
        "how can we make this work together", "what feels most useful right now"
    ],

    "Ineffective": [
        "let’s not dwell on that", "let’s move on", "you should just", "you need to", "why don’t you just",
        "just focus on the positive", "don't overthink it", "it’s not that bad", "everyone goes through this",
        "sounds like you’re overreacting", "it’s time to move on", "that’s not important right now",
        "focus on the task", "let’s not get emotional", "we don’t need to talk about that", "I’m sure you’ll be fine",
        "don’t dwell on the past", "just get over it", "this isn’t worth your time", "maybe you're overthinking it",
        "you just need to be more positive", "that’s in the past now", "let’s stay solution-focused",
        "it doesn’t help to think about it like that", "why don’t you focus on what’s good", "don’t let this get to you",
        "you should be grateful", "maybe try not to dwell on that", "let’s focus on moving forward",
        "why not just let it go", "there’s no point in thinking about that", "you shouldn’t let this bother you",
        "don’t take it so seriously", "you’re making a big deal out of this", "try to see the bright side",
        "let’s just keep going", "maybe you’re taking this too personally", "other people have it worse",
        "you’ll get over it", "it’s probably not as bad as you think", "you’re stronger than this",
        "just push through", "you can’t change the past", "try not to think about it so much",
        "maybe you’re being too sensitive", "why are you still thinking about this", "stop worrying about it",
        "just distract yourself", "you need to move on", "that’s not worth your energy", "just try harder",
        "you don’t need to feel that way", "you’re making this harder than it is", "don’t overanalyze this",
        "let’s not waste time on that", "you should know better than to feel this way"
    ]
}


# 📝 Summarization Code

## Clean transcripts

In [ ]:
import pandas as pd
import re
import contractions

# Define the cleaning function
def clean_text(text):
    # 1. Ensure the input is a string
    if not isinstance(text, str):
        text = str(text) if pd.notnull(text) else ""

    # 2. Remove timestamps like [00:15] or [1:15:30]
    text = re.sub(r'\[\d{1,2}:\d{2}(?::\d{2})?\]', '', text)

    # 3. Remove bracketed annotations (square or round), e.g., [laughs], (sigh)
    text = re.sub(r'\[.*?\]', '', text)  # Square brackets
    text = re.sub(r'\(.*?\)', '', text)  # Round brackets

    # 4. Expand contractions (e.g., "won’t" → "will not") need for some models
    text = contractions.fix(text)

    # 5. Remove ellipses and repeated punctuation
    text = re.sub(r'\.{2,}', '.', text)         # Replace "..." with "."
    text = re.sub(r'!{2,}', '!', text)          # Replace multiple ! with one
    text = re.sub(r'\?{2,}', '?', text)         # Replace multiple ? with one

    # 6. Normalize spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text



# Apply cleaning function to each speaker's text
Qdata['ClientClean'] = Qdata['Client'].apply(clean_text)
Qdata['TherapistClean'] = Qdata['Therapist'].apply(clean_text)

# Done! You now have cleaned columns ready for further analysis or modeling.


## key summarizing code

v6 focuses on three turns...see original v5 for full code and discussion. 3 and 4 turns seemed to achieve highest accuracy; one was useless. two was so so




In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import openai
import os

# ✅ Set your OpenAI API key from environment variables
openai.api_key = os.getenv("OPENAI_API_KEY")

# ✅ Choose the LLM model to use (GPT-3.5 or GPT-4)
llm_model = "gpt-3.5-turbo-1106"
# llm_model = "gpt-4-turbo"  # Uncomment this to use GPT-4

# ✅ Create output columns if they don't already exist in the dataframe
if "Summary_3Turn" not in Qdata.columns:
    Qdata["Summary_3Turn"] = pd.NA  # Stores the model's generated summary
if "RawText_3Turn" not in Qdata.columns:
    Qdata["RawText_3Turn"] = pd.NA  # Stores the actual 1–3 turn window

# ✅ Iterate through each session (grouped by participant ID)
for session_id, group in tqdm(Qdata.groupby("ID"), desc="Generating 3-Turn Summaries"):
    group_sorted = group.sort_values("Turn").copy()  # Sort each session chronologically
    idxs = group_sorted.index.tolist()  # List of index positions within this session

    # ✅ Loop through each turn in the session
    for i in range(len(group_sorted)):
        idx = idxs[i]

        # ✅ Skip if summary already exists for this row
        if pd.notnull(Qdata.at[idx, "Summary_3Turn"]):
            continue

        # ✅ Create a rolling window of 1–3 turns ending at the current turn
        # For i = 0, only 1 turn; i = 1, two turns; i ≥ 2, full 3-turn window
        turns = []
        for j in range(max(0, i - 2), i + 1):  # max(0, i - 2) handles early rows
            r = group_sorted.iloc[j]
            c_text = r["ClientClean"] if pd.notnull(r["ClientClean"]) else ""
            t_text = r["TherapistClean"] if pd.notnull(r["TherapistClean"]) else ""
            turns.append(f"Client: {c_text}\nTherapist: {t_text}".strip())  # Format the turn
        joined_turns = "\n\n".join(turns)  # Join turns with spacing

        # ✅ Save the raw 1–3-turn window to the dataframe
        Qdata.at[idx, "RawText_3Turn"] = joined_turns

        # ✅ Create the prompt for the model
        prompt = f"""
You are analyzing a 3-turn therapy exchange. Summarize the key psychological themes or shifts in this interaction. Avoid quoting. Use 1–2 sentences.

### Turns:
{joined_turns}
""".strip()

        # ✅ Call OpenAI API and store the model’s response
        try:
            response = openai.chat.completions.create(
                model=llm_model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0,
                max_tokens=150
            )
            Qdata.at[idx, "Summary_3Turn"] = response.choices[0].message.content.strip()
        except Exception as e:
            print(f"⚠️ Error on turn {i+1} in session {session_id}: {e}")
            Qdata.at[idx, "Summary_3Turn"] = ""

# ✅ Export the completed dataframe to Excel
Qdata.to_excel(output_file, index=False)
print(f"✅ 3-Turn Summaries saved to: {output_file}")


Generating 3-Turn Summaries:   0%|          | 0/53 [00:00<?, ?it/s]

✅ 3-Turn Summaries saved to: /content/drive/MyDrive/Colab Notebooks/datafiles/Synthetic_ratings.xlsx


# 📂 Load file with summaries

In [ ]:
import zipfile

print(zipfile.is_zipfile(output_file))  # Should be True for real .xlsx files


True


In [ ]:
import pandas as pd


# Load the Excel file (reads the first sheet by default)
SummarizedFile = pd.read_excel(output_file)

# Display the first few rows
SummarizedFile.head()
num_rows = SummarizedFile.shape[0]
print(f"Number of rows: {num_rows}")



Number of rows: 707


# 📚 Multicategory Classification psych

##window determination. Three turns

pilot work  with borutalwith half data set suggests that three turns better than two which is better than 1. Four may be best, but we don't wnat window too large so will settle on 3.

single catagory classification very poor. almost never produced important feature


In [ ]:
import pandas as pd
import openai
import os
import json
import re
from tqdm.notebook import tqdm

# =====================================================
# 🔧 CONFIGURATION — change settings here, not below
# =====================================================
start_fresh = False  # Set to True to reprocess all rows
llm_model = "gpt-3.5-turbo-1106"  # or "gpt-4"
#llm_model = "gpt-4o"

output_file = "/content/drive/MyDrive/Colab Notebooks/datafiles/Synthetic_ratings.xlsx"

fields_to_classify = [
    "Summary_3Turn", "RawText_3Turn"
]

# ✅ Set API key
openai.api_key = os.getenv("OPENAI_API_KEY")

# ✅ Load Qdata from the defined path
Qdata = pd.read_excel(output_file)

# ✅ Efficiently add all missing classification and status columns
new_cols = {}
for field in fields_to_classify:
    for cat in Psych_categories:
        full_col = f"{field}_{cat.replace(' ', '')}_full"
        if full_col not in Qdata.columns or start_fresh:
            new_cols[full_col] = 0.0
    for cat in ProcessFocusCategories:
        short_col = f"{field}_{cat.replace(' ', '')}_short"
        if short_col not in Qdata.columns or start_fresh:
            new_cols[short_col] = 0.0
    for suffix in ["full", "short"]:
        status_col = f"{field}_status_{suffix}"
        if status_col not in Qdata.columns or start_fresh:
            new_cols[status_col] = ""

# ✅ Add new columns to Qdata with correct dtypes
if new_cols:
    float_cols = {k: v for k, v in new_cols.items() if not k.endswith(("_status_full", "_status_short"))}
    status_cols = {k: v for k, v in new_cols.items() if k.endswith(("_status_full", "_status_short"))}

    float_df = pd.DataFrame(float_cols, index=Qdata.index).astype(float)
    status_df = pd.DataFrame(status_cols, index=Qdata.index).astype("string")

    Qdata = pd.concat([Qdata, float_df, status_df], axis=1)
    Qdata = Qdata.copy()

# ✅ Safe JSON parser with fallback fix
def safe_parse_json(content):
    try:
        return json.loads(content), "ok"
    except json.JSONDecodeError:
        try:
            fixed = re.sub(r'([{,])\s*([A-Za-z_][A-Za-z0-9_ ]*)\s*:', r'\1 "\2":', content)
            return json.loads(fixed), "fallback"
        except Exception:
            return None, "fail"

# ✅ Classification function with robust parsing and clarified labels
def classify_with_gpt(text, category_dict, field_name, suffix, row_idx):
    prompt = (
        "You are an expert in evaluating psychological processes in therapy conversations.\n"
        "For each category, read its definition, then rate how relevant it is to the text using a score from 0 (not relevant) to 10 (completely relevant).\n"
        "Return a JSON dictionary where the keys are category names and the values are float scores.\n\n"
        "Example: {\"Motivation\": 6.5, \"Affect\": 2.0}\n\n"
    )

    for name, definition in category_dict.items():
        prompt += f"### Category: {name}\n"
        prompt += f"### Definition:\n{definition.strip()}\n\n"

    input_label = "Verbatim Therapy Text" if "RawText" in field_name else "Therapy Summary"
    prompt += f"### {input_label}:\n{text.strip()}"

    try:
        response = openai.chat.completions.create(
            model=llm_model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            max_tokens=500
        )
        result_text = response.choices[0].message.content.strip()
        parsed, status = safe_parse_json(result_text)

        if parsed is None:
            raise ValueError("Parsing failed.")

        Qdata.at[row_idx, f"{field_name}_status_{suffix}"] = str("ok" if status == "ok" else "fallback")
        return parsed

    except Exception as e:
        print(f"⚠️ Error on {field_name}_{suffix} row {row_idx}: {e}\nRaw response:\n{result_text if 'result_text' in locals() else 'N/A'}")
        Qdata.at[row_idx, f"{field_name}_status_{suffix}"] = str("fail")
        return {key: 0.0 for key in category_dict}

# ✅ Main classification loop
for idx in tqdm(Qdata.index, desc="Classifying rows"):
    for field in fields_to_classify:
        text = Qdata.at[idx, field]
        if pd.notnull(text):
            full_status = Qdata.at[idx, f"{field}_status_full"]
            short_status = Qdata.at[idx, f"{field}_status_short"]

            if start_fresh or full_status != "ok":
                full_scores = classify_with_gpt(text, Psych_categories, field, "full", idx)
                for cat, score in full_scores.items():
                    Qdata.at[idx, f"{field}_{cat.replace(' ', '')}_full"] = float(score)

            if start_fresh or short_status != "ok":
                short_scores = classify_with_gpt(text, ProcessFocusCategories, field, "short", idx)
                for cat, score in short_scores.items():
                    Qdata.at[idx, f"{field}_{cat.replace(' ', '')}_short"] = float(score)

    Qdata.to_excel(output_file, index=False)

print("✅ All classifications complete and saved.")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/datafiles/Synthetic_ratings.xlsx'

# 🧲 Embedding-Based Classification psych



In [ ]:
import pandas as pd
import openai
import os
from tqdm.notebook import tqdm
from sklearn.metrics.pairwise import cosine_similarity

# ===========================================
# 🔧 CONFIGURATION
# ===========================================
start_fresh = False  # Set to True to reprocess all fields
embedding_model = "text-embedding-ada-002"
output_file = "/content/drive/MyDrive/Colab Notebooks/datafiles/Synethetic_ratings.xlsx"

fields_to_embed = [
    "Summary_3Turn", "RawText_3Turn"
]

# ✅ Set API key
openai.api_key = os.getenv("OPENAI_API_KEY")

# ✅ Load data
Qdata = pd.read_excel(output_file)

# ✅ Category embeddings
category_names = list(Psych_categories.keys())
category_texts = list(Psych_categories.values())

def get_embedding(text):
    try:
        response = openai.embeddings.create(
            input=text,
            model=embedding_model
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"⚠️ Embedding error: {e}")
        return [0.0] * 1536  # Return zero vector on failure

category_embeddings = [get_embedding(text) for text in category_texts]

# ✅ Add new columns if needed
for field in fields_to_embed:
    for cat in category_names:
        col_name = f"{field}_{cat.replace(' ', '')}_embed"
        if col_name not in Qdata.columns or start_fresh:
            Qdata[col_name] = None
    status_col = f"{field}_status_embed"
    if status_col not in Qdata.columns or start_fresh:
        Qdata[status_col] = ""

# ✅ Classification loop
def rate_embedding_similarity(text):
    try:
        emb = get_embedding(text)
        sims = cosine_similarity([emb], category_embeddings)[0]
        return {cat: round(score, 4) for cat, score in zip(category_names, sims)}
    except Exception as e:
        print(f"⚠️ Error computing similarity: {e}")
        return {cat: 0.0 for cat in category_names}

for idx in tqdm(Qdata.index, desc="Embedding-based classification"):
    for field in fields_to_embed:
        text = Qdata.at[idx, field]
        if pd.notnull(text):
            status_col = f"{field}_status_embed"
            if start_fresh or Qdata.at[idx, status_col] != "ok":
                result = rate_embedding_similarity(text)
                for cat, score in result.items():
                    Qdata.at[idx, f"{field}_{cat.replace(' ', '')}_embed"] = score
                Qdata.at[idx, status_col] = "ok"

    # ✅ Save after every row (safer on long runs)
    Qdata.to_excel(output_file, index=False)

print("✅ Embedding-based classification complete.")


Embedding-based classification:   0%|          | 0/707 [00:00<?, ?it/s]

✅ Embedding-based classification complete.


# ⚙️ Rule-Based Classification psych

---




In [ ]:
import pandas as pd
import re
from tqdm.notebook import tqdm

# ===========================================
# 🔧 CONFIGURATION
# ===========================================
start_fresh = False  # Set to True to reprocess all fields
output_file = "/content/drive/MyDrive/Colab Notebooks/datafiles/Synethetic_ratings.xlsx"

fields_to_score = [
    "Summary_3Turn", "RawText_3Turn"
]

# ✅ Load data
Qdata = pd.read_excel(output_file)

# ✅ Define rule-based scoring function
def rule_based_score(text, category_keywords):
    matches = sum(1 for kw in category_keywords if re.search(rf"\b{re.escape(kw)}\b", text.lower()))
    return min(10, matches * 2)  # Scaled: 2 points per match, capped at 10

# ✅ Add new columns if needed
for field in fields_to_score:
    for cat in Psych_keywords:
        col_name = f"{field}_{cat.replace(' ', '')}_rule"
        if col_name not in Qdata.columns or start_fresh:
            Qdata[col_name] = None
    status_col = f"{field}_status_rule"
    if status_col not in Qdata.columns or start_fresh:
        Qdata[status_col] = ""

# ✅ Classification loop
for idx in tqdm(Qdata.index, desc="Rule-Based Classification"):
    for field in fields_to_score:
        text = Qdata.at[idx, field]
        if pd.notnull(text):
            status_col = f"{field}_status_rule"
            if start_fresh or Qdata.at[idx, status_col] != "ok":
                for cat, keywords in Psych_keywords.items():
                    col_name = f"{field}_{cat.replace(' ', '')}_rule"
                    score = rule_based_score(text, keywords)
                    Qdata.at[idx, col_name] = score
                Qdata.at[idx, status_col] = "ok"

    # ✅ Save after every row (safer for long runs)
    Qdata.to_excel(output_file, index=False)

print("✅ Rule-based classification complete.")


Rule-Based Classification:   0%|          | 0/707 [00:00<?, ?it/s]

✅ Rule-based classification complete.


# 🚨👥🔥 SOCIAL PROCESSES 🔥👥🚨




## 🧑‍🤝‍🧑🗂️ Multi-Category Classification: Social

<h3>🔍 Outputs: Overview of Columns Created</h3>

<p><strong>Fields Evaluated Per Row:</strong></p>
<ul>
  <li><code>Summary_3Turn</code> – model-written summary of the full exchange</li>
  <li><code>RawText_3Turn</code> – raw transcript of the exchange</li>
  <li><code>Client</code> – summary of the client's portion</li>
  <li><code>Therapist</code> – summary of the therapist's portion</li>
</ul>

<p>Each field is rated on five core social process categories:</p>
<ul>
  <li><strong>Relational Content Focus</strong></li>
  <li><strong>Understanding</strong></li>
  <li><strong>Interpersonal Effectiveness</strong></li>
  <li><strong>Collaboration</strong></li>
  <li><strong>Ineffective</strong></li>
</ul>

<p>Ratings are produced using two types of category definitions:</p>
<ul>
  <li><strong>Full</strong> – based on <code>Social_categories</code></li>
  <li><strong>Short</strong> – based on <code>Brief_Social_categories</code></li>
</ul>

<h4>📌 Column Naming Convention</h4>
<p>Each output column is named using the following structure:</p>
<pre><code>[FieldName]_[CategoryNameNoSpaces]_[full|short]</code></pre>

<p><strong>Examples:</strong></p>
<ul>
  <li><code>Client_RelationalContentFocus_full</code></li>
  <li><code>Therapist_Understanding_short</code></li>
  <li><code>Summary_3Turn_Collaboration_full</code></li>
  <li><code>RawText_3Turn_Ineffective_short</code></li>
</ul>

<h4>📌 Status Columns</h4>
<p>Each field also has two status columns that indicate classification status:</p>
<pre><code>[FieldName]_status_full
[FieldName]_status_short</code></pre>

<p><strong>Examples:</strong></p>
<ul>
  <li><code>Client_status_full</code></li>
  <li><code>Therapist_status_short</code></li>
</ul>

<h4>📌 Value Format</h4>
<ul>
  <li>Each score is a float from <strong>0.0 to 10.0</strong></li>
  <li>Status values: <code>"ok"</code>, <code>"fallback"</code>, or <code>"fail"</code></li>
</ul>

<h4>📦 Summary</h4>
<ul>
  <li><strong>20 rating columns</strong> per row (4 fields × 5 categories)</li>
  <li><strong>8 status columns</strong> (4 fields × 2 suffixes)</li>
</ul>






##methods section

To support the classification of social processes in therapy sessions, we prepared four distinct text fields for large language model (LLM) analysis: raw client utterances, raw therapist utterances, three-turn rolling windows of therapist–client dialogue, and brief summaries of those windows. For each turn in a session, the dataset preserved the verbatim text of the therapist and client contributions as separate fields. These single-turn utterances were classified directly without any modification. In contrast, the rolling window fields were constructed to capture broader contextual patterns: at each conversational point, up to three sequential therapist–client exchanges (i.e., turns) were
combined into a single block of text (RawText_3Turn). These rolling windows allowed for a temporally grounded understanding of interactional dynamics.

Each RawText_3Turn entry was paired with a GPT-generated summary (Summary_3Turn) that described key psychological shifts or themes in one to two sentences. Summaries were produced using a standardized prompt that instructed the model (GPT-3.5-turbo) to analyze the interaction and extract central themes without quoting directly. The full 3-turn transcript and the resulting summary were stored alongside each corresponding entry. All four fields—raw client, raw therapist, rolling 3-turn text, and rolling summary—were then classified using two sets of category definitions (full and brief), allowing us to compare model ratings across raw and summarized representations. This multi-layered design enabled us to examine how different levels of abstraction and context contribute to the identification of social processes in therapy.

In [ ]:
import pandas as pd
import openai
import os
import json
import re
from tqdm.notebook import tqdm

# =====================================================
# 🔧 CONFIGURATION
# =====================================================
start_fresh = False  # Set to True to reprocess all rows
#llm_model = "gpt-4o"
llm_model = "gpt-3.5-turbo-1106"


import os

target_filename = "synthetic_ratings.xlsx"

def find_file_path_by_name(filename_exact):
    search_dir = "/content/drive/My Drive"
    for root, dirs, files in os.walk(search_dir):
        for file in files:
            if file.lower() == filename_exact.lower():
                filepath = os.path.join(root, file)
                print(f"✅ File found: {filepath}")
                return filepath
    raise FileNotFoundError(f"❌ File named '{filename_exact}' not found in Google Drive.")

output_file = find_file_path_by_name(target_filename)


fields_to_classify = ["Summary_3Turn", "RawText_3Turn", "Client", "Therapist"]

# ✅ Set API key
openai.api_key = os.getenv("OPENAI_API_KEY")

# ✅ Load data
Qdata = pd.read_excel(output_file)




# =====================================================
# ✅ ADD NEW COLUMNS IF NEEDED
# =====================================================
new_cols = {}
for field in fields_to_classify:
    for cat in Social_categories:
        full_col = f"{field}_{cat.replace(' ', '')}_full"
        if full_col not in Qdata.columns or start_fresh:
            new_cols[full_col] = 0.0
    for cat in Brief_Social_categories:
        short_col = f"{field}_{cat.replace(' ', '')}_short"
        if short_col not in Qdata.columns or start_fresh:
            new_cols[short_col] = 0.0
    for suffix in ["full", "short"]:
        status_col = f"{field}_status_{suffix}"
        if status_col not in Qdata.columns or start_fresh:
            new_cols[status_col] = ""

if new_cols:
    float_cols = {k: v for k, v in new_cols.items() if not k.endswith(("_status_full", "_status_short"))}
    status_cols = {k: v for k, v in new_cols.items() if k.endswith(("_status_full", "_status_short"))}

    float_df = pd.DataFrame(float_cols, index=Qdata.index).astype(float)
    status_df = pd.DataFrame(status_cols, index=Qdata.index).astype("string")

    Qdata = pd.concat([Qdata, float_df, status_df], axis=1)
    Qdata = Qdata.copy()


# =====================================================
# ✅ UTILITY FUNCTIONS
# =====================================================
def safe_parse_json(content):
    try:
        return json.loads(content), "ok"
    except json.JSONDecodeError:
        try:
            fixed = re.sub(r'([{,])\s*([A-Za-z_][A-Za-z0-9_ ]*)\s*:', r'\1 "\2":', content)
            return json.loads(fixed), "fallback"
        except Exception:
            return None, "fail"


def classify_with_gpt(text, category_dict, field_name, suffix, row_idx):
    prompt = (
        "You are an expert in evaluating therapist and client social processes in therapy conversations.\n"
        "For each category, read its definition, then rate how relevant it is to the text using a score from 0 (not relevant) to 10 (completely relevant).\n"
        "Return a JSON dictionary where the keys are category names and the values are float scores.\n\n"
        "Example: {\"Relational Content Focus\": 6.5, \"Understanding\": 2.0}\n\n"
    )

    for name, definition in category_dict.items():
        prompt += f"### Category: {name}\n"
        prompt += f"### Definition:\n{definition.strip()}\n\n"

    input_label = "Verbatim Therapy Text" if "RawText" in field_name else "Therapy Summary"
    prompt += f"### {input_label}:\n{text.strip()}"

    try:
        response = openai.chat.completions.create(
            model=llm_model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            max_tokens=500
        )
        result_text = response.choices[0].message.content.strip()
        parsed, status = safe_parse_json(result_text)

        if parsed is None:
            raise ValueError("Parsing failed.")

        Qdata.at[row_idx, f"{field_name}_status_{suffix}"] = str("ok" if status == "ok" else "fallback")
        return parsed

    except Exception as e:
        print(f"⚠️ Error on {field_name}_{suffix} row {row_idx}: {e}\nRaw response:\n{result_text if 'result_text' in locals() else 'N/A'}")
        Qdata.at[row_idx, f"{field_name}_status_{suffix}"] = str("fail")
        return {key: 0.0 for key in category_dict}


# =====================================================
# ✅ MAIN CLASSIFICATION LOOP
# =====================================================
for idx in tqdm(Qdata.index, desc="Classifying rows"):
    for field in fields_to_classify:
        text = Qdata.at[idx, field]
        if pd.notnull(text):
            full_status = Qdata.at[idx, f"{field}_status_full"]
            short_status = Qdata.at[idx, f"{field}_status_short"]

            if start_fresh or full_status != "ok":
                full_scores = classify_with_gpt(text, Social_categories, field, "full", idx)
                for cat, score in full_scores.items():
                    Qdata.at[idx, f"{field}_{cat.replace(' ', '')}_full"] = float(score)

            if start_fresh or short_status != "ok":
                short_scores = classify_with_gpt(text, Brief_Social_categories, field, "short", idx)
                for cat, score in short_scores.items():
                    Qdata.at[idx, f"{field}_{cat.replace(' ', '')}_short"] = float(score)

    Qdata.to_excel(output_file, index=False)

print("✅ All classifications complete and saved.")


✅ File found: /content/drive/My Drive/Colab Notebooks/datafiles/Synthetic_ratings.xlsx


Classifying rows:   0%|          | 0/707 [00:00<?, ?it/s]

⚠️ Error on Client_full row 445: Parsing failed.
Raw response:
```json
{
  "Relational Content Focus": 8.0,
  "Understanding": 9.0,
  "Interpersonal effectiveness": 7.5,
  "Collaboration": 6.5,
  "Ineffective": 4.0
}
```
⚠️ Error on Therapist_short row 705: Parsing failed.
Raw response:
```json
{
  "Relational Content Focus": 8.0,
  "Understanding": 7.5,
  "Interpersonal effectiveness": 9.0,
  "Collaboration": 8.0,
  "Ineffective": 2.0
}
```
✅ All classifications complete and saved.


## 🧑‍🤝‍🧑🗂️ Multi-Category Classification: Social 📊🧑‍🤝‍🧑 Rule-Based Classification for Social Keywords

fuzzy matching used because unlikely to nail, say, invalidating response exactly. fuzzy matching delutes with length so split into sentences. exact matching does not (above).

In [ ]:
import pandas as pd
import os
import re
from rapidfuzz import fuzz
from tqdm.notebook import tqdm

# ✅ Locate and load your Excel file
target_filename = "synthetic_ratings.xlsx"

def find_file_path_by_name(filename_exact):
    search_dir = "/content/drive/My Drive"
    for root, dirs, files in os.walk(search_dir):
        for file in files:
            if file.lower() == filename_exact.lower():
                filepath = os.path.join(root, file)
                print(f"✅ File found: {filepath}")
                return filepath
    raise FileNotFoundError(f"❌ File named '{filename_exact}' not found in Google Drive.")

output_file = find_file_path_by_name(target_filename)
Qdata = pd.read_excel(output_file)
print("✅ Qdata loaded with shape:", Qdata.shape)

# ✅ Define sentence-based fuzzy rule-based scoring
def rule_based_score_sentence_fuzzy(text, category_keywords, threshold=75):
    text = text.lower()
    sentences = re.split(r'[.!?]', text)  # Rough sentence splitter
    matches = 0
    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence:
            continue
        for kw in category_keywords:
            if kw.lower() in sentence:
                matches += 1
            elif fuzz.partial_ratio(sentence, kw.lower()) >= threshold:
                matches += 1
    return min(10, matches * 2)  # Scaled: 2 points per match, capped at 10

# ✅ Configuration
start_fresh = False  # Set True to reprocess existing columns
fields_to_score = ["Client", "Therapist"]  # Columns containing text

# ✅ Add new columns if needed
for field in fields_to_score:
    for cat in Social_keywords:
        col_name = f"{field}_{cat.replace(' ', '')}_rule"
        if col_name not in Qdata.columns or start_fresh:
            Qdata[col_name] = None
    status_col = f"{field}_status_rule"
    if status_col not in Qdata.columns or start_fresh:
        Qdata[status_col] = ""

# ✅ Classification loop
for idx in tqdm(Qdata.index, desc="Rule-Based Classification (Sentences)"):
    for field in fields_to_score:
        text = Qdata.at[idx, field]
        if pd.notnull(text):
            status_col = f"{field}_status_rule"
            if start_fresh or Qdata.at[idx, status_col] != "ok":
                for cat, keywords in Social_keywords.items():
                    col_name = f"{field}_{cat.replace(' ', '')}_rule"
                    score = rule_based_score_sentence_fuzzy(text, keywords)
                    Qdata.at[idx, col_name] = score
                Qdata.at[idx, status_col] = "ok"

# ✅ Save updated file
Qdata.to_excel(output_file, index=False)
print("✅ Fuzzy rule-based classification (sentence level) complete.")


✅ File found: /content/drive/My Drive/Colab Notebooks/datafiles/Synthetic_ratings.xlsx
✅ Qdata loaded with shape: (707, 30)


Rule-Based Classification (Sentences):   0%|          | 0/707 [00:00<?, ?it/s]

✅ Fuzzy rule-based classification (sentence level) complete.
